# This Notebook is for capstone project

In [1]:
import pandas as pd
import numpy as np

print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [2]:
from bs4 import BeautifulSoup
import requests
import csv
import json
from pandas.io.json import json_normalize

# Data Scraping

In [3]:
# data request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(data, 'lxml')

postalCodeList = []
boroughList = []
neighborhoodList = []

In [4]:
#append data
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text.rstrip('\n'))
        boroughList.append(cells[1].text.rstrip('\n'))
        neighborhoodList.append(cells[2].text.rstrip('\n'))

In [5]:
toronto_df = pd.DataFrame({"Postal Code": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

toronto_df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [6]:
#drop cells with Borough not assigned
toronto_dropped = toronto_df[toronto_df.Borough != 'Not assigned'].reset_index(drop=True)
toronto_dropped.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [7]:
toronto_group = toronto_dropped.groupby(['Postal Code', 'Borough'], as_index=False).agg(lambda x: ", ".join(x))
toronto_group['Neighborhood'] = toronto_group['Neighborhood'].str.replace('/', ',')
toronto_group.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
for index, row in toronto_group.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]

In [14]:
toronto_group

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [16]:
print(toronto_group.shape)

(103, 3)


# GeoLocation

In [17]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd
import folium
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
!conda install -c conda-forge lxml --yes
import requests
import lxml.html as lh
from sklearn.cluster import KMeans
print("Libraries imported.")

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\syasha\anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.9.2                |   py38haa244fe_0         3.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    python_abi-3.8             |           1_cp38           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0
  python_abi         conda-forge/win-64::python_abi-3.8-1_cp38

The following packag

In [19]:
toronto_group["Latitude"] = ""
toronto_group["Longitude"] = ""

In [20]:
toronto_group

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",,
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",,
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",,
3,M1G,Scarborough,Woburn,,
4,M1H,Scarborough,Cedarbrae,,
...,...,...,...,...,...
98,M9N,York,Weston,,
99,M9P,Etobicoke,Westmount,,
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",,
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",,


In [22]:
df1 = toronto_group.loc[0:25]
df2 = toronto_group.loc[26:50]
df3 = toronto_group.loc[51:75]
df4 = toronto_group.loc[76:102]

In [23]:
# Need to drop those Neighborhood that the geocode does not find
to_drop_unknown = []
geolocator = Nominatim(user_agent="ny_explorer")
for index, row in df1.iterrows():
    address = row['Neighborhood'] + ', Toronto'
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
        df1.loc[index, 'Latitude'] = latitude
        df1.loc[index, 'Longitude'] = longitude
    except AttributeError:
        print('Cannot do: {}, will drop index: {}'.format(address, index))
        to_drop_unknown.append(index)

The geograpical coordinate of Malvern, Rouge, Toronto are 43.8091955, -79.2217008.


C:\Users\syasha\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-23-ff0f5384fede>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.loc[index, 'Latitude'] = latitude
<ipython-input-23-ff0f5384fede>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.loc[index, 'Longitude'] = longitude


Cannot do: Rouge Hill, Port Union, Highland Creek, Toronto, will drop index: 1
The geograpical coordinate of Guildwood, Morningside, West Hill, Toronto are 43.7907925, -79.19369853569341.
The geograpical coordinate of Woburn, Toronto are 43.7598243, -79.2252908.
The geograpical coordinate of Cedarbrae, Toronto are 43.75646655, -79.22669244258802.
The geograpical coordinate of Scarborough Village, Toronto are 43.7437422, -79.2116324.
Cannot do: Kennedy Park, Ionview, East Birchmount Park, Toronto, will drop index: 6
Cannot do: Golden Mile, Clairlea, Oakridge, Toronto, will drop index: 7
Cannot do: Cliffside, Cliffcrest, Scarborough Village West, Toronto, will drop index: 8
Cannot do: Birch Cliff, Cliffside West, Toronto, will drop index: 9
Cannot do: Dorset Park, Wexford Heights, Scarborough Town Centre, Toronto, will drop index: 10
Cannot do: Wexford, Maryvale, Toronto, will drop index: 11
The geograpical coordinate of Agincourt, Toronto are 43.7853531, -79.2785494.
Cannot do: Clarks C

In [24]:
geolocator = Nominatim(user_agent="ny_explorer2")
for index, row in df2.iterrows():
    address = row['Neighborhood'] + ', Toronto'
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
        df2.loc[index, 'Latitude'] = latitude
        df2.loc[index, 'Longitude'] = longitude
    except AttributeError:
        print('Cannot do: {}, will drop index: {}'.format(address, index))
        to_drop_unknown.append(index)

The geograpical coordinate of Don Mills, Toronto are 43.775347, -79.3459439.


<ipython-input-24-bdd38c9d5d55>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.loc[index, 'Latitude'] = latitude
<ipython-input-24-bdd38c9d5d55>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.loc[index, 'Longitude'] = longitude


The geograpical coordinate of Don Mills, Toronto are 43.775347, -79.3459439.
Cannot do: Bathurst Manor, Wilson Heights, Downsview North, Toronto, will drop index: 28
The geograpical coordinate of Northwood Park, York University, Toronto are 43.7706699, -79.50334760664649.
The geograpical coordinate of Downsview, Toronto are 43.7492988, -79.462248.
The geograpical coordinate of Downsview, Toronto are 43.7492988, -79.462248.
The geograpical coordinate of Downsview, Toronto are 43.7492988, -79.462248.
The geograpical coordinate of Downsview, Toronto are 43.7492988, -79.462248.
The geograpical coordinate of Victoria Village, Toronto are 43.732658, -79.3111892.
The geograpical coordinate of Parkview Hill, Woodbine Gardens, Toronto are 43.7145315, -79.30948979272151.
The geograpical coordinate of Woodbine Heights, Toronto are 43.69992, -79.319279.
The geograpical coordinate of The Beaches, Toronto are 43.6710244, -79.296712.
The geograpical coordinate of Leaside, Toronto are 43.7047983, -79.

In [26]:
geolocator = Nominatim(user_agent="ny_explorer3")
for index, row in df3.iterrows():
    address = row['Neighborhood'] + ', Toronto'
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
        df3.loc[index, 'Latitude'] = latitude
        df3.loc[index, 'Longitude'] = longitude
    except AttributeError:
        print('Cannot do: {}, will drop index: {}'.format(address, index))
        to_drop_unknown.append(index)

Cannot do: St. James Town, Cabbagetown, Toronto, will drop index: 51
The geograpical coordinate of Church and Wellesley, Toronto are 43.6708625, -79.37279241253721.


C:\Users\syasha\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-26-989a6405256f>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3.loc[index, 'Latitude'] = latitude
<ipython-input-26-989a6405256f>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3.loc[index, 'Longitude'] = longitude


Cannot do: Regent Park, Harbourfront, Toronto, will drop index: 53
Cannot do: Garden District, Ryerson, Toronto, will drop index: 54
The geograpical coordinate of St. James Town, Toronto are 43.6694032, -79.3727041.
The geograpical coordinate of Berczy Park, Toronto are 43.64798435, -79.37539591138858.
The geograpical coordinate of Central Bay Street, Toronto are 43.6556436, -79.3837005.
Cannot do: Richmond, Adelaide, King, Toronto, will drop index: 58
Cannot do: Harbourfront East, Union Station, Toronto Islands, Toronto, will drop index: 59
Cannot do: Toronto Dominion Centre, Design Exchange, Toronto, will drop index: 60
Cannot do: Commerce Court, Victoria Hotel, Toronto, will drop index: 61
The geograpical coordinate of Bedford Park, Lawrence Manor East, Toronto are 43.7373876, -79.4109253.
The geograpical coordinate of Roselawn, Toronto are 43.7074896, -79.4154939.
The geograpical coordinate of Forest Hill North & West, Forest Hill Road Park, Toronto are 43.701028, -79.4255998.
Cann

In [27]:
geolocator = Nominatim(user_agent="ny_explorer4")
for index, row in df4.iterrows():
    address = row['Neighborhood'] + ', Toronto'
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
        df4.loc[index, 'Latitude'] = latitude
        df4.loc[index, 'Longitude'] = longitude
    except AttributeError:
        print('Cannot do: {}, will drop index: {}'.format(address, index))
        to_drop_unknown.append(index)

The geograpical coordinate of Dufferin, Dovercourt Village, Toronto are 43.6602019, -79.4357191.


<ipython-input-27-fd771227c2c1>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4.loc[index, 'Latitude'] = latitude
<ipython-input-27-fd771227c2c1>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4.loc[index, 'Longitude'] = longitude


Cannot do: Little Portugal, Trinity, Toronto, will drop index: 77
Cannot do: Brockton, Parkdale Village, Exhibition Place, Toronto, will drop index: 78
Cannot do: North Park, Maple Leaf Park, Upwood Park, Toronto, will drop index: 79
Cannot do: Del Ray, Mount Dennis, Keelsdale and Silverthorn, Toronto, will drop index: 80
The geograpical coordinate of Runnymede, The Junction North, Toronto are 43.6517026, -79.4759978.
The geograpical coordinate of High Park, The Junction South, Toronto are 43.6538668, -79.4668644.
The geograpical coordinate of Parkdale, Roncesvalles, Toronto are 43.6412811, -79.4515993.
The geograpical coordinate of Runnymede, Swansea, Toronto are 43.6517776, -79.4759227.
Cannot do: Queen's Park, Ontario Provincial Government, Toronto, will drop index: 85
Cannot do: Canada Post Gateway Processing Centre, Toronto, will drop index: 86
Cannot do: Business reply mail Processing Centre, South Central Letter Processing Plant Toronto, Toronto, will drop index: 87
Cannot do: N

In [32]:

print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)

(26, 5)
(25, 5)
(25, 5)
(27, 5)


In [33]:
t_group = df1.append(df2, ignore_index = True)
t_group = t_group.append(df3, ignore_index = True)
t_group = t_group.append(df4, ignore_index = True)

In [31]:
t_group

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.8092,-79.2217
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",,
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7908,-79.1937
3,M1G,Scarborough,Woburn,43.7598,-79.2253
4,M1H,Scarborough,Cedarbrae,43.7565,-79.2267
...,...,...,...,...,...
98,M9N,York,Weston,43.7002,-79.5162
99,M9P,Etobicoke,Westmount,43.6936,-79.521
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",,
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",,


In [34]:
clean_df = t_group.drop(to_drop_unknown)
clean_df['Latitude'].replace('', np.nan, inplace=True)
clean_df.dropna(subset=['Latitude'], inplace=True)
clean_df.shape

(58, 5)

In [35]:
clean_df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.809196,-79.2217
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.790793,-79.1937
3,M1G,Scarborough,Woburn,43.759824,-79.2253
4,M1H,Scarborough,Cedarbrae,43.756467,-79.2267
5,M1J,Scarborough,Scarborough Village,43.743742,-79.2116
12,M1S,Scarborough,Agincourt,43.785353,-79.2785
15,M1W,Scarborough,"Steeles West, L'Amoreaux West",43.802655,-79.3079
16,M1X,Scarborough,Upper Rouge,43.804930,-79.1658
17,M2H,North York,Hillcrest Village,43.681695,-79.4257
19,M2K,North York,Bayview Village,43.769197,-79.3767


In [40]:
Toronto_df = clean_df[clean_df['Borough'].str.contains("Toronto")].reset_index(drop=True)
Toronto_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.671024,-79.2967
1,M4K,East Toronto,"The Danforth West, Riverdale",43.667044,-79.3478
2,M4M,East Toronto,Studio District,43.649585,-79.3907
3,M4N,Central Toronto,Lawrence Park,43.729199,-79.4033
4,M4P,Central Toronto,Davisville North,43.697936,-79.3973


In [44]:
Toronto_df.drop_duplicates(subset ="Postal Code",keep = "first", inplace = True)
Toronto_df.reset_index(drop=True) 
print(Toronto_df.shape)

(19, 5)


In [37]:
# define Foursquare Credentials and Version
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
toronto_df_new = Toronto_df.copy()

Your credentails:
CLIENT_ID: 2WUBGK1NICYDJF3MDCY3TFIYCVEAOLUPPPTYAWIEHR532SYU
CLIENT_SECRET:2LNDZ0TH3GRHDPEDVQBAQCO4ZP5XTABL1J02GDQ5WWGCAPT3


In [45]:
# get nearby Venues
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Postal Code'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,        CLIENT_SECRET,        VERSION,        lat,        long,        radius,         LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post,             borough,            neighborhood,            lat,             long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [46]:

# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(785, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.671024,-79.296712,Kew Gardens,43.669038,-79.298538,Park
1,M4E,East Toronto,The Beaches,43.671024,-79.296712,Mastermind Toys,43.671453,-79.293971,Toy / Game Store
2,M4E,East Toronto,The Beaches,43.671024,-79.296712,The Ten Spot,43.670340,-79.299363,Nail Salon
3,M4E,East Toronto,The Beaches,43.671024,-79.296712,Sanna's Farmacia,43.670929,-79.295969,Juice Bar
4,M4E,East Toronto,The Beaches,43.671024,-79.296712,Kew-Balmy Beach,43.667372,-79.295312,Beach


In [47]:
# one hot encoding to do cluster
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['PostalCode'] = venues_df['PostalCode'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(785, 176)


,PostalCode,Borough,Neighborhoods,African Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Trail,Tree,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped.head()

(19, 176)


,PostalCode,Borough,Neighborhoods,African Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Trail,Tree,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,M4E,East Toronto,The Beaches,0.0,0.0,0.0,0.00,0.0,0.00,0.000000,...,0.000000,0.022222,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.0,0.00
1,M4K,East Toronto,"The Danforth West, Riverdale",0.0,0.0,0.0,0.00,0.0,0.00,0.033333,...,0.000000,0.000000,0.000000,0.00,0.066667,0.00,0.0,0.166667,0.0,0.00
2,M4M,East Toronto,Studio District,0.0,0.0,0.0,0.02,0.0,0.02,0.010000,...,0.000000,0.000000,0.000000,0.01,0.040000,0.01,0.0,0.000000,0.0,0.02
3,M4N,Central Toronto,Lawrence Park,0.0,0.0,0.0,0.00,0.0,0.00,0.039216,...,0.000000,0.000000,0.019608,0.00,0.000000,0.00,0.0,0.000000,0.0,0.00
4,M4P,Central Toronto,Davisville North,0.0,0.0,0.0,0.00,0.0,0.00,0.027778,...,0.027778,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.0,0.00


In [51]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

(19, 13)


,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Beach,Breakfast Spot,Nail Salon,Park,Pizza Place,Bar,Japanese Restaurant,Pub,Restaurant,Sandwich Place
1,M4K,East Toronto,"The Danforth West, Riverdale",Vietnamese Restaurant,Café,Fast Food Restaurant,Vegetarian / Vegan Restaurant,Grocery Store,Bakery,Caribbean Restaurant,Bar,Coffee Shop,Light Rail Station
2,M4M,East Toronto,Studio District,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Pizza Place,Clothing Store,Cosmetics Shop,Japanese Restaurant,Hotel,Taco Place,Movie Theater
3,M4N,Central Toronto,Lawrence Park,Sushi Restaurant,Coffee Shop,Bakery,Italian Restaurant,Bank,Pub,Fast Food Restaurant,Asian Restaurant,Cosmetics Shop,Camera Store
4,M4P,Central Toronto,Davisville North,Sushi Restaurant,Italian Restaurant,Coffee Shop,Park,Pub,Convenience Store,Ice Cream Shop,Mexican Restaurant,Gastropub,Ramen Restaurant


# Clustering

In [53]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = toronto_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 2, 2, 2, 1, 0, 0, 0])

In [55]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
toronto_merged = toronto_df_new.copy()
# add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("Postal Code"), on="Postal Code")
toronto_merged.rename(columns = {'BoroughLatitude':'Latitude', 'BoroughLongitude':'Longitude'}, inplace = True) 
print(toronto_merged.shape)
toronto_merged.head() # check the last columns!

KeyError: "None of ['Postal Code'] are in the columns"

In [57]:
map_clusters  = folium.Map(location=[latitude, longitude], zoom_start=12)
# set color scheme for the clusters
x = np.arange(kclusters)
rainbow = [    'red',    'blue',    'orange',    'darkgreen',    'darkblue',    'black']
# add markers to map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postal Code'], toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters